<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/TAPT_seq_bn_pretrain_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This is 100 epoch pretraining of the TAPT for amazon helpfulness on seq_bn.

# Settings
* LR of 0.0001
* No warmup
* Set batch size to 21, gradient_accumulation_steps=2, ~6 reviews per row = 252 batch size (paper has 256)
* Adapter - seq_bn
  * torch_compile=False (Because no adapter)
* early_stopping_patience=10




# Setup

In [ ]:
from time import time
start_time = time()

In [ ]:
# from google.colab import drive
# drive.mount("/content/drive")
# %cd '/content/drive/MyDrive/AdaptOrDie'

In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters
# !pip install scikit-learn

In [ ]:
!pip install pynvml

In [ ]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")

print_gpu_utilization()

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

# Pretraining

In [ ]:
from datasets import load_dataset

dataset_name = "BigTMiami/amazon_helpfulness_TAPT_pretraining_dataset"

dataset = load_dataset(dataset_name, split=["test","dev"])

print(dataset)

In [ ]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

In [ ]:
# THIS MUST BE SET TO MATCH  pretraining_adapter_type
from adapters import SeqBnConfig

adapter_config = SeqBnConfig()
adapter_name = "dapt_tapt_seq_bn"
adapter_hub_name = f"pretrain_{adapter_name}_adpater"

In [ ]:
print(adapter_hub_name)

In [ ]:
dapt_adapter_name = "jgrc3/RobertaDAPT_adapters_pfeiffer_reDo"

In [ ]:
# Add a new adapter
loaded_name = model.load_adapter(dapt_adapter_name, config=adapter_config)

# Set the adapter to be used for training
model.train_adapter(loaded_name)

print(f"Loaded Adapter Name: {loaded_name}")

In [ ]:
summary = model.adapter_summary()
print(summary)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [ ]:
from transformers import DataCollatorForLanguageModeling

pretraining_data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [ ]:
from transformers import TrainingArguments

pretraining_args = TrainingArguments(
    output_dir="dapt_tapt_seq_bn_helpfulness_pretrained_model",
    learning_rate=0.0001, # Paper for TAPT training
    per_device_train_batch_size=21,
    per_device_eval_batch_size=21,
    gradient_accumulation_steps=2, # Creates effective batch of 22 * 2 * 6 = 252
    num_train_epochs=100,
    weight_decay=0.01,
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2, # Saves latest 2 checkpoints
    # torch_compile=True,  # Much Faster
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    logging_strategy="epoch", # Is default
    metric_for_best_model="eval_loss", # Is default
    load_best_model_at_end=True,
    report_to="tensorboard",
)

In [ ]:
from transformers import EarlyStoppingCallback

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=10)

In [ ]:
from adapters import AdapterTrainer

trainer = AdapterTrainer(
    model=model,
    args=pretraining_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["dev"],
    data_collator=pretraining_data_collator,
    callbacks=[early_stopping_callback],
)

In [ ]:
pretrain_results = trainer.train()
print(pretrain_results)

In [ ]:
eval_results = trainer.evaluate(dataset["test"])
print(eval_results)

In [ ]:
model.push_adapter_to_hub(
    adapter_hub_name,
    adapter_name,
    datasets_tag=dataset_name,
    overwrite_adapter_card=True,
)

In [ ]:
trainer.push_to_hub()

In [ ]:
end_time = time()
total_time = end_time - start_time
print(f"Total Time: {total_time/60:.0f} minutes OR {total_time:.0f} seconds")

In [ ]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()